In [1]:
import qbraid
from qbraid.devices._utils import SUPPORTED_VENDORS

In [2]:
def device_wrapper_input(vendor):
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        for device in SUPPORTED_VENDORS[vendor][provider]:
            if provider != "D-Wave":
                data = (device, provider, vendor)
                input_list.append(data)
    return input_list

In [3]:
qbraid.get_devices()

Software Vendor    Device Provider    Device Name
-----------------  -----------------  -----------------------------
AWS                AWS                simulator_statevector
AWS                AWS                simulator_densitymatrix
AWS                AWS                simulator_tensornetwork
AWS                AWS                local_simulator_default
AWS                D-Wave             DW_2000Q_6
AWS                D-Wave             Advantage_system1
AWS                IonQ               ionQdevice
AWS                Rigetti            Aspen-9
Google             Google             local_simulator_default
Google             Google             local_simulator_densitymatrix
IBM                IBMQ               armonk
IBM                IBMQ               belem
IBM                IBMQ               bogota
IBM                IBMQ               lima
IBM                IBMQ               manila
IBM                IBMQ               quito
IBM                IBMQ               sa

In [28]:
ibm_wrapper = qbraid.device_wrapper("simulator_aer", "Aer", vendor="IBM")

In [29]:
from qiskit import QuantumCircuit

In [30]:
circuit = QuantumCircuit(1)  # Create a qiskit circuit
circuit.h(0)
circuit.y(0)
circuit.measure_all()

In [31]:
qbraid_job = ibm_wrapper.run(circuit, shots=100)

In [32]:
qbraid_job.done()

True

In [33]:
qbraid_job.result().data()

{'counts': {'0x0': 53, '0x1': 47}}

In [34]:
qbraid_job.vendor_jlo

In [35]:
from qiskit.providers.job import Job

In [36]:
isinstance(qbraid_job.vendor_jlo, Job)

True